In [2]:
import pandas as pd

def reallocate_weights(stock_weights):
    #Reallocate stock weights to ensure they remain within the range of 3% to 10%.

    # Parameters:
    #     stock_weights (pd.DataFrame): A DataFrame with columns ['Stock', 'Weight'], where Weight is the original weight.

    # Returns:
    #     pd.DataFrame: Updated DataFrame with adjusted weights meeting the criteria.
    
    # Define constraints
    MIN_WEIGHT = 0.03
    MAX_WEIGHT = 0.10

    # Ensure weights are normalized to sum to 1
    stock_weights['Weight'] = stock_weights['Weight'] / stock_weights['Weight'].sum()

    # Iteratively adjust weights
    while True:
        # Identify stocks that violate the constraints
        below_min = stock_weights[stock_weights['Weight'] < MIN_WEIGHT]
        above_max = stock_weights[stock_weights['Weight'] > MAX_WEIGHT]
        # Redistribution if weights are below the minimum and others are above the maximum
        if not below_min.empty and not above_max.empty:
            excess_weight = above_max['Weight'].sum() - MAX_WEIGHT * len(above_max)
            needed_weight = MIN_WEIGHT * len(below_min) - below_min['Weight'].sum()
            # Redistribute weights from stocks above MAX_WEIGHT to those below MIN_WEIGHT
            reallocation = min(excess_weight, needed_weight)
            stock_weights.loc[above_max.index, 'Weight'] -= reallocation * stock_weights.loc[above_max.index, 'Weight'] / above_max['Weight'].sum()
            stock_weights.loc[below_min.index, 'Weight'] += reallocation * stock_weights.loc[below_min.index, 'Weight'] / below_min['Weight'].sum()
        # Redistribution if no stock is above MAX_WEIGHT but some are below MIN_WEIGHT
        elif not below_min.empty and above_max.empty:
            available_weight = stock_weights[~stock_weights.index.isin(below_min.index)]['Weight'].sum() - MAX_WEIGHT * len(stock_weights[~stock_weights.index.isin(below_min.index)])
            needed_weight = MIN_WEIGHT * len(below_min) - below_min['Weight'].sum()
            # Redistribute weights from eligible stocks proportionally
            reallocation = min(available_weight, needed_weight)
            eligible_stocks = stock_weights[~stock_weights.index.isin(below_min.index)]
            stock_weights.loc[eligible_stocks.index, 'Weight'] -= reallocation * stock_weights.loc[eligible_stocks.index, 'Weight'] / eligible_stocks['Weight'].sum()
            stock_weights.loc[below_min.index, 'Weight'] += reallocation * stock_weights.loc[below_min.index, 'Weight'] / below_min['Weight'].sum()
        # Redistribution if excess weight still exists
        elif not above_max.empty:
            excess_weight = above_max['Weight'].sum() - MAX_WEIGHT * len(above_max)
            eligible_stocks = stock_weights[stock_weights['Weight'] <= MAX_WEIGHT]
            stock_weights.loc[above_max.index, 'Weight'] -= excess_weight * stock_weights.loc[above_max.index, 'Weight'] / above_max['Weight'].sum()
            stock_weights.loc[eligible_stocks.index, 'Weight'] += excess_weight * stock_weights.loc[eligible_stocks.index, 'Weight'] / eligible_stocks['Weight'].sum()
        else:
            # Exit the loop if all weights meet the constraints
            break
    # Ensure weights are normalized after adjustments
    stock_weights['Weight'] = stock_weights['Weight'] / stock_weights['Weight'].sum()
    return stock_weights

# Example usage
data = {
    'Stock': ['StockA', 'StockB', 'StockC', 'StockD', 'StockE'],
    'Weight': [0.02, 0.12, 0.08, 0.15, 0.10]  # Example initial weights
}
stock_weights_df = pd.DataFrame(data)

# Apply reallocation
adjusted_weights = reallocate_weights(stock_weights_df)

# Display the result
print(adjusted_weights)